# Evaluating the fine tuned model

### Needed packages and imports

In [1]:
!pip install -r requirements.txt

  Obtaining dependency information for docling[tesserocr]~=2.8.3 from https://files.pythonhosted.org/packages/02/e9/8d81e497365224e2ea80ce0b625f1e9339d736a8a7f7c2224c6f56be3131/docling-2.8.3-py3-none-any.whl.metadata
  Obtaining dependency information for einops==0.8.0 from https://files.pythonhosted.org/packages/44/5a/f0b9ad6c0a9017e62d4735daaeb11ba3b6c009d69a26141b258cd37b5588/einops-0.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for langchain==0.3.11 from https://files.pythonhosted.org/packages/ba/4a/26620afcff880f6058756786d9b858d348ac29c815e44f57b6c2c07bf86d/langchain-0.3.11-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community==0.3.11 from https://files.pythonhosted.org/packages/dc/c2/aeb43e7f879d429df40f742228dba747d790655f3eb0e2082b7d9854f0fd/langchain_community-0.3.11-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-openai==0.2.12 from https://files.pythonhosted.org/packages/cd/01/7a0259929474c47cadec

### Model inference parameters

The parameters to the fine tuned model.

In [2]:
import requests
import os
import yaml
import json
import re
import time
import pandas as pd
import torch

from typing import Iterator
from pathlib import Path
from openai import OpenAI

from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document as LCDocument

from langchain_openai import ChatOpenAI
from langchain_community.llms import VLLMOpenAI
from langchain_milvus import Milvus
from langchain_text_splitters import RecursiveCharacterTextSplitter

from docling.document_converter import DocumentConverter

def replace_special_char(original_str):
    return re.sub(r"[^\w]", "_", original_str)


In [3]:
import os

from dotenv import load_dotenv

load_dotenv()

False

In [4]:
MAX_TOKENS=2048
TEMPERATURE=0.00

### Milvus connection info

Defaults to local db

In [5]:
MILVUS_URI = os.getenv("MILVUS_URI", "./milvus_llm_judge_eval.db")
MILVUS_USERNAME = os.getenv("MILVUS_USERNAME", "")
MILVUS_PASSWORD = os.getenv("MILVUS_PASSWORD", "")
MILVUS_COLLECTION = os.getenv("MILVUS_COLLECTION", "my_org_documents")

## Sanity check model

In [6]:
def create_llm(testing_config):
    if testing_config.get("model_type") == "openai":
        print("Creating OpenAI model")
        return ChatOpenAI(
            openai_api_key=re.sub(r"\s+", "", testing_config["api_key"]),
            model=testing_config["model_name"],
            streaming=False
        )
    print("Creating VLLM model")
    return VLLMOpenAI(
        openai_api_key=re.sub(r"\s+", "", testing_config["api_key"]),
        openai_api_base=testing_config["endpoint_url"], #https://model...com/v1
        model_name=testing_config["model_name"],
        temperature=0.00,
        max_tokens=2048,
        streaming=False
    )

def qna_request(llm, template_str, question):
    num_retries = 1
    for attempt in range(num_retries):
        try:
            prompt = PromptTemplate.from_template(template_str)
            chain = prompt | llm | StrOutputParser()
            answer = chain.invoke({"question": question})        
            print(answer)
            return answer.strip()
        except Exception as e:
            print(f"Request failed: {e}")
            if attempt + 1 < num_retries:
                print(f"Retrying in 5 seconds...")
                time.sleep(5)
            else:
                return ""


In [7]:
with open("llm_config.yaml", "r") as f:
    llm_config = yaml.safe_load(f)
    
llm = create_llm(llm_config["testing_config"][0])
template_str = llm_config["testing_config"][0]["qna_template"]

question = "What is Artificial Intelligence?"
qna_request(llm, template_str, question)

Creating VLLM model
Artificial Intelligence (AI) is a multidisciplinary field of study within computer science that focuses on creating intelligent machines capable of performing tasks that would typically require human intelligence. These tasks include learning from experience, understanding natural language, recognizing patterns, solving problems, and making decisions. AI systems can be categorized into two main types: narrow or weak AI, designed to perform a specific task, and general or strong AI, which can perform any intellectual task that a human being can do.


'Artificial Intelligence (AI) is a multidisciplinary field of study within computer science that focuses on creating intelligent machines capable of performing tasks that would typically require human intelligence. These tasks include learning from experience, understanding natural language, recognizing patterns, solving problems, and making decisions. AI systems can be categorized into two main types: narrow or weak AI, designed to perform a specific task, and general or strong AI, which can perform any intellectual task that a human being can do.'

## Creating an Milvus DB with documents

## Initial index creation and document ingestion

#### Load pdfs

In [8]:
class DoclingPDFLoader(BaseLoader):

    def __init__(self, file_path: str | list[str]) -> None:
        self._file_paths = file_path if isinstance(file_path, list) else [file_path]
        self._converter = DocumentConverter()

    def lazy_load(self) -> Iterator[LCDocument]:
        for source in self._file_paths:
            dl_doc = self._converter.convert(source).document
            text = dl_doc.export_to_markdown()
            yield LCDocument(page_content=text)

In [9]:
pdf_folder_path = "../data_preparation/document_collection"
file_paths = [str(path) for path in Path(pdf_folder_path).rglob('*.pdf')]
file_paths

['../data_preparation/document_collection/ny_budget/fy25cp-en.pdf',
 '../data_preparation/document_collection/ny_policies/its-p19-005-open-source/its-p19-005-open-source.pdf',
 '../data_preparation/document_collection/ny_policies/nys-s16-001-new-york-state-universal-web-navigation/nys-s16-001-new-york-state-universal-web-navigation.pdf',
 '../data_preparation/document_collection/ny_policies/its-p10-003-its-telecommuting-policy/its-p10-003-its-telecommuting-policy.pdf',
 '../data_preparation/document_collection/ny_policies/nys-p08-003-domain-names-for-state-government/nys-p08-003-domain-names-for-state-government.pdf',
 '../data_preparation/document_collection/ny_policies/nys-s14-009_mobile_device_security/nys-s14-009_mobile_device_security.pdf',
 '../data_preparation/document_collection/ny_policies/its-p04-005-surplus-and-disposal-of-its-equipment-furniture-and-vehicles/its-p04-005-surplus-and-disposal-of-its-equipment-furniture-and-vehicles.pdf',
 '../data_preparation/document_collect

In [10]:
loader = DoclingPDFLoader(file_path=file_paths)

#### Split documents into chunks with some overlap

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=100,
)

docs = loader.load()
splits = text_splitter.split_documents(docs)
splits[0]

/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 9 files: 100%|██████████| 9/9 [00:05<00:00,  1.55it/s]


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Document(metadata={}, page_content='<!-- image -->\n\n<!-- image -->')

#### Create the index and ingest the documents

In [14]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

model_kwargs = {"trust_remote_code": True, "device": device}
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1.5",
    model_kwargs=model_kwargs,
    show_progress=True
)

db = Milvus(
    embedding_function=embeddings,
    connection_args={
        "uri": MILVUS_URI,
        "user": MILVUS_USERNAME, 
        "password": MILVUS_PASSWORD
    },
    collection_name=MILVUS_COLLECTION,
    auto_id=True,
    drop_old=True
)


/tmp/ipykernel_381/3330881644.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A n

In [15]:
loaded = db.add_documents(splits)
print(f"{len(loaded)} documents loaded.")

Batches: 100%|██████████| 123/123 [00:55<00:00,  2.22it/s]


3909 documents loaded.


#### Test vector DB search

In [16]:
query = "Who are the funding partners for the State's transformative infrastructure projects?"
docs_with_score = db.similarity_search_with_score(query)

Batches: 100%|██████████| 1/1 [00:00<00:00, 77.33it/s]


In [17]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  146.87289428710938
-  Invests in projects to improve the State's transit systems, modernize airports, and rebuild infrastructure to upgrade the State's transportation network. Several projects are expected to be financed from multiple funding sources and administered by public authorities (e.g., MTA, PANYNJ) outside of the State budget. Funding partners for these projects include the State, local governments, the Federal government, public authorities, and private entities. The FY 2025 Enacted Budget supports the State share of funding for these projects. Major infrastructure projects underway with funding partners are shown in the table below.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  208.61204528808594
TRANSFORMATIVE INFRASTRUCTURE PROJECTS (billions of dollars)

|     

#### Test out RAG request

In [18]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def rag_request(llm, template_str, question):
    num_retries = 1
    for attempt in range(num_retries):
        try:
            prompt = PromptTemplate.from_template(template_str)
            rag_chain = (
                    {"context": retriever | format_docs, "question": RunnablePassthrough()}
                    | prompt
                    | llm
                    | StrOutputParser()
            )
            response = rag_chain.invoke(question)
            return response.strip()
        except Exception as e:
            print(f"Request failed: {e}")
            if attempt < num_retries:
                print(f"Retrying in 5 seconds...")
                time.sleep(5)
            else:
                return ""

In [19]:
llm = create_llm(llm_config["testing_config"][0])
template_str = llm_config["testing_config"][0]["rag_template"]

question = "Who are the funding partners for the State's transformative infrastructure projects?"
result = rag_request(llm, template_str, question)
result

Creating VLLM model


Batches: 100%|██████████| 1/1 [00:00<00:00, 78.63it/s]


"The funding partners for the State's transformative infrastructure projects include the State, local governments, the Federal government, public authorities, and private entities. The table in the context provides a breakdown of the total project costs, the State's share, and the share of other funding partners. For example, the Gateway Tunnel Project has a total project cost of $16 billion, with the State's share being $1.3 billion and the share of other funding partners being $14.7 billion."

## Generate Answers

### Use qna.yaml, csv, jsonl to create some questions and ground truth answers

We create a pandas dataframe with the columns `question` and `ground_truth`
- create a csv file in the correct (default is "ground_truth") directory with the columns `question` and `ground_truth`

- qna.yaml files can be taken as written from data_preparation and converted to the appropriate format.  

In [31]:
with open("llm_config.yaml", "r") as f:
    llm_config = yaml.safe_load(f)

directory = "ground_truth"
output_directory = llm_config.get("name", "output")
os.makedirs(output_directory, exist_ok=True)
qround_truth_df = pd.DataFrame(columns=["question", "ground_truth"])

for file_path in Path(directory).rglob('*.csv'):
    # print(file_path)
    csv_df = pd.read_csv(file_path)
    print(f"{file_path}: {csv_df.shape[0]} questions")
    qround_truth_df = pd.concat([qround_truth_df, csv_df], ignore_index=True)

for file_path in Path(directory).rglob('*.jsonl'):
    # print(file_path)
    jsonl_df = pd.read_json(file_path, orient="records", lines=True)
    print(f"{file_path}: {jsonl_df.shape[0]} questions")
    qround_truth_df = pd.concat([qround_truth_df, jsonl_df], ignore_index=True)

qna_list = []

for file_path in Path(directory).rglob('*.yaml'):
    with open(file_path) as file:
        qna = yaml.load(file, Loader=yaml.FullLoader)
        for seed_example in qna["seed_examples"]:
            for questions_and_answers in seed_example["questions_and_answers"]:
                qna_list.append({
                    "question": questions_and_answers["question"].strip(),
                    "ground_truth": questions_and_answers["answer"].strip()
                })
        print(f"{file_path}: {len(qna_list)} questions")

ground_truth_df = pd.concat([qround_truth_df, pd.DataFrame(qna_list)], ignore_index=True)
ground_truth_df = ground_truth_df.drop_duplicates(subset=["question"])
print(f"{ground_truth_df.shape[0]} total unique questions")

ground_truth_df.to_json(f"{output_directory}/ground_truth.jsonl", orient="records", lines=True)
ground_truth_df.head()

ground_truth/qna.csv: 11 questions
ground_truth/qna.jsonl: 13 questions
ground_truth/qna.yaml: 15 questions
39 total unique questions


,question,ground_truth
0,What does an employee need to provide in an ap...,An employee needs to provide a request in writ...
1,Who is on the Telecommuting Appeal Board?,The telecommuting appeal board consists of a r...
2,How long do employees have to wait until they ...,Employees who have been removed from the Telec...
3,Should SE websites have sub-domain addresses?,SE websites should use short URLs over a separ...
4,When should SE domain names use acronyms?,SE websites should avoid the use of acronyms u...


## Get responses from each of the available models with and without RAG

In [32]:
import yaml


with open("llm_config.yaml", "r") as f:
    llm_config = yaml.safe_load(f)

output_directory = llm_config.get("name", "output")
os.makedirs(output_directory, exist_ok=True)   
    
qna_df = pd.read_json(f"{output_directory}/ground_truth.jsonl", orient="records", lines=True)

for testing_config in llm_config["testing_config"]:
    answers = qna_df.copy()
    answers["answer"] = ""
    answers["rag_answer"] = ""
    llm = create_llm(testing_config)
    for index, row in answers.iterrows():
        question = row["question"]
        print(index, question[:40])
        if testing_config.get("qna_template"):
            answer = qna_request(llm, testing_config.get("qna_template"), question)
            if answer:
                print("QnA Answer: " + answer[:40])
            answers.at[index, "answer"] = answer
        if testing_config.get("rag_template"):
            answer = rag_request(llm, testing_config.get("rag_template"), question)
            if answer:
                print("RAG Answer: " + answer[:40])
            answers.at[index, "rag_answer"] = answer
    testing_config_name = replace_special_char(testing_config["name" or "model_name"])
    answers.to_json(f"{output_directory}/{testing_config_name}_answers.jsonl", orient="records", lines=True)
    # answers.to_csv(f"{output_directory}/{base_filename}_answers.csv")

Creating VLLM model
0 What does an employee need to provide in
In an appeal to the Telecommuting Appeal Board, an employee typically needs to provide a written request with reasons for disagreement with the determination made by the Telecommuting Program Coordinator. This request should include any new information or evidence that supports the employee's case. The employee may also have the opportunity to present their case in person during a hearing before the Appeal Board. It's important to note that the specific requirements for an appeal may vary depending on the organization, so it's always a good idea to check the relevant policies and guidelines for the most accurate information.
QnA Answer: In an appeal to the Telecommuting Appeal


Batches: 100%|██████████| 1/1 [00:00<00:00, 79.41it/s]


RAG Answer: In an appeal to the Telecommuting Appeal
1 Who is on the Telecommuting Appeal Board
The Telecommuting Appeal Board typically consists of three members. These members are selected from various departments within the organization to ensure a fair and unbiased review of telecommuting appeals. The specific departments and roles of the members can vary depending on the organization, but they are chosen to bring a diverse set of perspectives and expertise to the decision-making process.
QnA Answer: The Telecommuting Appeal Board typically


Batches: 100%|██████████| 1/1 [00:00<00:00, 79.77it/s]


RAG Answer: The Telecommuting Appeal Board consists 
2 How long do employees have to wait until
Employees must wait six months before submitting a new application for the telecommuting program. This waiting period allows them to demonstrate their commitment and performance in the current telecommuting arrangement before applying again.
QnA Answer: Employees must wait six months before su


...

RAG Answer: The context provided does not include sp
37 What are the projected appropriations fr


Batches: 100%|██████████| 1/1 [00:00<00:00, 85.47it/s]


RAG Answer: The context provided does not include sp
38 What is the total projected appropriatio


Batches: 100%|██████████| 1/1 [00:00<00:00, 72.00it/s]


RAG Answer: The total projected appropriation on pro


## Grade responses using Judge Model

In [33]:
import yaml

with open("llm_config.yaml", "r") as f:
    llm_config = yaml.safe_load(f)

output_directory = llm_config.get("name", "output")
os.makedirs(output_directory, exist_ok=True)

In [34]:
from langchain.prompts import PromptTemplate

scoring_template_str = llm_config["judge"].get("template")
assert scoring_template_str
SCORING_PROMPT = PromptTemplate.from_template(scoring_template_str)

In [35]:
judge_client = OpenAI(api_key=llm_config["judge"]["api_key"])
judge_model_name = llm_config["judge"]["model_name"]


def score_request(question, answer, reference_answer):
    messages = [
        {
            "role": "user",
            "content": SCORING_PROMPT.format(
                question=question,
                answer=answer,
                reference_answer=reference_answer
            )
        }
    ]

    completion = judge_client.chat.completions.create(
        model=judge_model_name,
        messages=messages,
        n=1,
        temperature=0.0,
        max_tokens=1024,
    )
    response_content = completion.choices[0].message.content
    # print(response_content)
    response_content = re.sub(r'^```json', '', response_content)
    response_content = re.sub(r'```$', '', response_content)
    # print(response_content)
    try:
        result = json.loads(response_content)
    except Exception as e:
        result = {"answer_quality": 0, "reasoning": "Error"}
        print("response_content:", response_content)
        print(f"An error occurred: {e}")

    score = result["answer_quality"]
    reasoning = result["reasoning"]
    return score, reasoning


In [36]:
for testing_config in llm_config["testing_config"]:
    testing_config_name = replace_special_char(testing_config["name" or "model_name"])
    print("-" * 80)
    print(testing_config_name)
    answers_filename = f"{output_directory}/{testing_config_name}_answers.jsonl"
    scores = pd.read_json(answers_filename, orient="records", lines=True)
    position = scores.columns.get_loc("answer")
    scores.insert(position + 1, "answer_score", "")
    scores.insert(position + 2, "answer_score_reasoning", "")
    position = scores.columns.get_loc("rag_answer")
    scores.insert(position + 1, "rag_answer_score", "")
    scores.insert(position + 2, "rag_answer_score_reasoning", "")

    for index, row in scores.iterrows():
        question = row["question"]
        answer = row["answer"]
        reference_answer = row["ground_truth"]
        print(index, question)
        if answer:
            score, reasoning = score_request(question, answer, reference_answer)
            scores.at[index, "answer_score"] = score
            scores.at[index, "answer_score_reasoning"] = reasoning
            print(answer[:40], score, reasoning[:40])
        rag_answer = row["rag_answer"]
        if rag_answer:
            score, reasoning = score_request(question, rag_answer, reference_answer)
            scores.at[index, "rag_answer_score"] = score
            scores.at[index, "rag_answer_score_reasoning"] = reasoning
            print(rag_answer[:40], score, reasoning[:40])
    judge_name = replace_special_char(judge_model_name)
    scores_filename = f"{output_directory}/{testing_config_name}_scores"
    scores.to_json(f"{scores_filename}.jsonl", orient="records", lines=True)
    scores.to_csv(f"{scores_filename}.csv", index=False)


--------------------------------------------------------------------------------
finetuned
0 What does an employee need to provide in an appeal to the Telecommuting Appeal board?
In an appeal to the Telecommuting Appeal 4 The reference answer specifies that an e
In an appeal to the Telecommuting Appeal 4 The reference answer specifies that an e
1 Who is on the Telecommuting Appeal Board?
The Telecommuting Appeal Board typically 3 The reference answer specifies that the 
The Telecommuting Appeal Board consists  5 The reference answer states that the Tel
2 How long do employees have to wait until they may submit a new application for the telecommuting program?
Employees must wait six months before su 4 The reference answer states that employe
Employees who have had their application 3 The reference answer states that employe
3 Should SE websites have sub-domain addresses?
Yes, SE (State Entity) websites should h 2 The reference answer suggests that SE we
Yes, SE websites should use short

## Create resulting score report CSV

In [37]:
with open("llm_config.yaml", "r") as f:
    llm_config = yaml.safe_load(f)

output_directory = llm_config.get("name", "output")
os.makedirs(output_directory, exist_ok=True)

judge_client = OpenAI(api_key=llm_config["judge"]["api_key"])
judge_model_name = llm_config["judge"]["model_name"]
judge_name = replace_special_char(judge_model_name)

summary_output_df = pd.DataFrame()

for testing_config in llm_config["testing_config"]:
    testing_config_name = replace_special_char(testing_config["name" or "model_name"])
    scores_filename = f"{output_directory}/{testing_config_name}_scores.jsonl"
    scores = pd.read_json(scores_filename, orient="records", lines=True)
    if testing_config.get("qna_template"):
        summary_output_df[f"{testing_config_name}_answer_score"] = scores["answer_score"]
    if testing_config.get("rag_template"):
        summary_output_df[f"{testing_config_name}_rag_answer_score"] = scores["rag_answer_score"]

average_row = summary_output_df.mean(axis=0, numeric_only=True)
print(average_row)
summary_output_df.loc[len(summary_output_df)] = average_row
question_indices = [f"Q{i+1}" for i in range(len(summary_output_df)-1)]
question_indices.append("Average")
summary_output_df.insert(0, 'question index', question_indices)

summary_filepath = f"{output_directory}/summary_scores"
# summary_output_df.to_json(f"{summary_filepath}.jsonl", orient="records", lines=True)
summary_output_df.to_csv(f"{summary_filepath}.csv", index=False)

finetuned_answer_score                      3.128205
finetuned_rag_answer_score                  3.974359
granite_3_0_8b_instruct_rag_answer_score    3.897436
dtype: float64


In [38]:
with pd.ExcelWriter(f"{output_directory}/{judge_name}_scores.xlsx") as writer:
    summary_output_df = pd.read_csv(f"{summary_filepath}.csv")
    summary_output_df.to_excel(writer, sheet_name="Summary", index=False)

    for testing_config in llm_config["testing_config"]:
        testing_config_name = replace_special_char(testing_config["name" or "model_name"])
        scores_filename = f"{output_directory}/{testing_config_name}_scores.jsonl"
        scores = pd.read_json(scores_filename, orient="records", lines=True)
        scores.to_excel(writer, sheet_name=f"{testing_config_name}_scores"[:30])